In [140]:
from officelib.xllib import *
from officelib.const import xlconst as xlc

from scripts.tools.redmine_cache import RedmineClient
import importlib
import jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter2 as iie
import collections

iie = importlib.reload(iie)

_client = None
_get_client()

def _get_client():
    global _client
    if _client is None:
        _client = RedmineClient()
    return _client
def _download_issues():
    _client = _get_client()
    return _client.get_filtered([('project.identifier', '==', 'pbssoftware')])

def get_issues():
    return _download_issues()

In [141]:
def _load_trm_inner(fp):
    xl = Excel()
    with screen_lock(xl):
        wb = xl.Workbooks.Open(fp)
        ws = wb.Worksheets(1)
        ret = ws.UsedRange.Value2
        wb.Close(False)
    xl.Quit()
    return ret
    
def load_trm(fp):
    data = _load_trm_inner(fp)
    ret = collections.defaultdict(set)
    for vi, frs, btested in data:
        
        # one of the requirements was entered
        # incorrectly in the spreadsheet or whatever,
        # so this fixes it
        if isinstance(frs, int):
            frs = "FRS"+str(frs)
        
        frs = frs.strip()
        vi = vi.strip()
        ret[frs].add(vi)
    return ret

In [142]:
xl = Excel()
with screen_lock(xl):
    wb = xl.Workbooks.Add()
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range
    c1 = cr("A1")
    c2 = c1.GetOffset(len(mout)-1, len(mout[0])-1)
    cr(c1, c2).Value2 = mout
    ws.Columns(1).AutoFit()
    ws.Columns(2).AutoFit()
    ws.Rows.AutoFit()
    wb.SaveAs("C:\\users\\nathan\\documents\\personal\\test\\3.0.8\\out\\design input VI matrix.xlsx", FileFormat=xlc.xlOpenXMLWorkbook)

In [143]:
iie = importlib.reload(iie)
import re
def parse_line(find, line):
    typenums = find(line)
    for typ, num in typenums:
        yield typ + num
    
covers_find = re.compile(r".*\[Covers\:(.*)\].*").match

def comment_to_line(comment):
    m = covers_find(comment)
    if m:
        return m.group(1)
    else:
        return ""

def parse_lvtest_file(fp):
    with open(fp, 'r', encoding="utf_16_le") as f:
        lines = f.read().splitlines()
    data = [line.split("\t") for line in lines]
    
    it = iter(data)
    allrefs = set()
    _, _, _, ref_find = iie._make_regex_ctx(["URS", "FRS", "SWDS", "SDS"])
    
    # section 1: metadata
    next(it)  # skip header
    for values in it:
        if len(values) == 1 and values[0] == "":
            break
        if values[0] == "Requirement ID":
            allrefs.update(parse_line(ref_find, values[1]))
        #print(repr(values))
    
    # section2: test cases & comments
    next(it)  # skip header
    for values in it:
        if len(values) == 1 and values[0] == "":
            break
        line = comment_to_line(values[1])
        allrefs.update(parse_line(ref_find, line))
        
    return allrefs

def extract_lvtest_files(fps):
    mtx = collections.defaultdict(set)
    for fp in fps:
        refs = parse_lvtest_file(fp)
        fn = os.path.split(fp)[1]
        for ref in refs:
            mtx[ref].add(fn)
    return mtx

def mtx_to_data(mtx, reqs):
    data = []
    for r in reqs:
        vis = mtx[r.tag]
        data.append((r.tag, "\n".join(vis)))
    return data
    
def data_to_xl(data):
    xl = Excel()
    with screen_lock(xl):
        wb = xl.Workbooks.Add()
        ws=wb.Worksheets(1)
        cells =ws.Cells
        cr = cells.Range
        c1 = cr("A1")
        c2 = c1.GetOffset(len(data)-1, len(data[0])-1)
        cr(c1, c2).Value2 = data
        ws.Columns(1).AutoFit()
        ws.Columns(2).AutoFit()
        ws.Rows.AutoFit()
        return wb

In [144]:
def relevant(i):
    return i.id == 4443 or (i.sprint_milestone.name == '3.0.8' and i.priority.name in ('High', 'Urgent', 'Immediate'))

iie = importlib.reload(iie)
issues = get_issues()
print("Parsing relevant issues")
relevant = [i for i in issues.values() if relevant(i)]
reqs = iie.IssuetrackerParser(["URS","FRS","SWDS"]).parse_all(relevant)

Parsing relevant issues


In [145]:
fd = "C:\\Users\\Nathan\\Documents\\PBS\\SW test\\LV 3.0.8\\r3.0release\\branches\\3.0.8\\iss4398\\Utilities\\Traceability Matrix\\Traceability Matrix (Reqs and VI).csv"
trm = load_trm(fd)
reqs = [r for r in reqs if not r.obs]
mout = []
for r in reqs:
    vis = trm[r.tag]
    mout.append((r.tag, "\n".join(vis)))

In [146]:
#fd = "C:\\users\\nathan\\documents\\personal\\test\\3.0.8"
basefd = "C:\\Users\\Nathan\\Documents\\PBS\\SW test\\LV 3.0.8\\"
files = [
    "r3.0release/branches/3.0.8/iss4398/Test/Unit Tests/Embedded Controller Code/Config/Load Bioreactor Configuration Unit Test 1.lvtest",
    "r3.0release/branches/3.0.8/iss4398/Test/Unit Tests/Embedded Controller Code/Control Modules/Aux Pumps Module Unit Test 1.lvtest",
    "r3.0release/branches/3.0.8/iss4398/Test/Unit Tests/Embedded Controller Code/Control Modules/SubVIs/Addition Pumps Unit Test 1.lvtest",
    "r3.0release/branches/3.0.8/iss4398/Test/Unit Tests/Web GUI/Webservice Command Handler SubVI/Set Base Pump for Web UI Unit Test 1.lvtest",
]
files = [os.path.normpath(f) for f in files]
fps = [os.path.join(basefd, f) for f in files]

In [147]:
mtx = extract_lvtest_files(fps)
data = mtx_to_data(mtx, reqs)
wb = data_to_xl(data)
wb.SaveAs(os.path.join("C:\\users\\nathan\\documents\\personal\\test\\3.0.8\\out\\", 
                       "design input lvtest matrix.xlsx"), FileFormat=xlc.xlOpenXMLWorkbook)